In [4]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

2.13.0


In [5]:
import tensorflow as tf
print(tf.__version__)  # 2.20.0


2.13.0


In [6]:
## loading the model
model = load_model("retention_risk_scoring_model.keras")

## loading the encoder and scaler
with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)

with open("onehot_encoder_geography.pkl", "rb") as f:
    onehot_encoder_geography = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [ ]:
## input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 35,
    "Tenure": 4,
    "Balance": 77000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [ ]:
## converting categorical features into numerical